# **IMPORTS/COMFIGIRAÇÕES**

In [1]:
import numpy as np  # Manipulação de matrizes
import pandas as pd  # Manipulação de dados tabulares
# Bibliotecs de visualização gráfica
import plotly.express as px
# Bibliotecas próprias
from ETL.Data_extraction import *  # Biblioteca própria para extração dos dados
from ETL.Data_transformation import *  # Biblioteca própria para transformação dos dados
# Biblioteca de filtro de notificações
import warnings



warnings.filterwarnings(action='ignore')
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
pd.options.display.float_format = lambda x: f'{x:,.2f}'
px.defaults.template = 'plotly_dark'

# **LOAD**

In [2]:
upgrade_data(fold='./RAW_DATAS')

Arquivo 2024 desatualizado, refazendo download.


In [3]:
df = pd.read_parquet('./DATASETS/summarized_data.parquet')
df.sample(5)

TIPO DE REPASSE,ESFERA,UF,COMPETÊNCIA,REPASSE,TOTAL AJUSTE,TOTAL LIQUIDO,CATEGORIA
146339,Municipal,RR,2014-01-01,ITCMD,0.00,"4,862.10",ITCMD
133197,Municipal,PR,2007-11-01,Ajuste,0.00,0.00,Outros
10864,Estadual,AP,2017-09-01,AFE_EC123,0.00,0.00,Outros
2976,Estadual,AC,2024-09-01,ICMS,0.00,0.00,ICMS
149622,Municipal,RS,2015-08-01,COUN_VAAR,0.00,0.00,Complementação VAAR


# **ANÁLISE DESCRITIVA**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163296 entries, 0 to 163295
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   ESFERA         163296 non-null  category      
 1   UF             163296 non-null  category      
 2   COMPETÊNCIA    163296 non-null  datetime64[ns]
 3   REPASSE        163296 non-null  category      
 4   TOTAL AJUSTE   163296 non-null  float64       
 5   TOTAL LIQUIDO  163296 non-null  float64       
 6   CATEGORIA      163296 non-null  category      
dtypes: category(4), datetime64[ns](1), float64(2)
memory usage: 4.4 MB


In [5]:
df.memory_usage()

Index                132
ESFERA            163420
UF                164584
COMPETÊNCIA      1306368
REPASSE           163964
TOTAL AJUSTE     1306368
TOTAL LIQUIDO    1306368
CATEGORIA         163684
dtype: int64

## Variáveis numéricas

In [6]:
num_vars = df.dtypes[(df.dtypes.values == 'float64')].index
num_vars

Index(['TOTAL AJUSTE', 'TOTAL LIQUIDO'], dtype='object', name='TIPO DE REPASSE')

In [7]:
df.describe()

TIPO DE REPASSE,COMPETÊNCIA,TOTAL AJUSTE,TOTAL LIQUIDO
count,163296,"163,296.00","163,296.00"
mean,2015-12-16 10:53:20,"68,091.21","14,763,894.63"
min,2007-01-01 00:00:00,"-213,329,018.65","-177,097,598.83"
25%,2011-06-23 12:00:00,0.00,0.00
50%,2015-12-16 12:00:00,0.00,"6,925.55"
75%,2020-06-08 12:00:00,0.00,"4,585,400.58"
max,2024-12-01 00:00:00,"696,137,219.82","2,431,147,109.50"
std,NaN,"4,550,686.55","70,659,385.27"


In [6]:
# # Tratamento de duplicados
# df[df.duplicated()]
# # Tratamento de valores ausentes
# df.loc[df.isnull().any(axis=1)]
# # Tratamento de valores inconsistentes ou despadronizados 
# ## Foram identificados valores negativos, sendo a maioria ajustes anteriores a 2020 ou contribuições da união, mas há dois casos com impostos, ITCDM e IPVA
# ## Não se sabe ainda o motivo, entretanto, para os valores finais, total liquido, encontran-se os valores informados pelo Fnde
# ## Não fora disponibilizado os metadados e deverá ser feita análise desses valores
# df.loc[df['TOTAL LIQUIDO'] < 0]
# # Tratamento de valores outliers
# ## 


## Variáveis categóricas

In [6]:
cat_vars = df.dtypes[df.dtypes == 'category'].index
cat_vars

Index(['ESFERA', 'UF', 'REPASSE', 'CATEGORIA'], dtype='object', name='TIPO DE REPASSE')

In [8]:
df.describe(include='category')

TIPO DE REPASSE,ESFERA,UF,REPASSE,CATEGORIA
count,163296,163296,163296,163296
unique,2,27,14,11
top,Estadual,AC,AFE_EC123,Outros
freq,81648,6048,11664,46656


In [9]:
df.ESFERA.unique()

['Estadual', 'Municipal']
Categories (2, object): ['Estadual', 'Municipal']

In [12]:
df.UF.unique()

['AC', 'AL', 'AM', 'AP', 'BA', ..., 'RS', 'SC', 'SE', 'SP', 'TO']
Length: 27
Categories (27, object): ['AC', 'AL', 'AM', 'AP', ..., 'SC', 'SE', 'SP', 'TO']

In [12]:
df.REPASSE.unique()

array(['AFE_EC123', 'Ajuste', 'COUN', 'COUN_VAAF', 'COUN_VAAR',
       'COUN_VAAT', 'FPE', 'FPM', 'ICMS', 'IPI', 'IPVA', 'ITCMD', 'ITR',
       'LC8796'], dtype=object)

x - Distribuição dos valores mensais (boxplot)

In [9]:
df_temp = df_e.copy()
df_temp = df_temp.resample('m').sum()

fig = px.box(data_frame=df_temp, x=['TOTAL LIQUIDO'], points='all', hover_name=df_temp.index)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y', xaxis_tickformat=',.2f')
fig.show()

x - Valor total dos repasses por ano

In [3]:
df_temp = df_e.copy()
df_temp = df_temp.resample(rule='y').sum(numeric_only=True)

fig = px.line(data_frame=df_temp,  y='TOTAL LIQUIDO', x=df_temp.index.year, title='Total de repasses efetuados aos estados por ano de 2007 a 09-2023', markers='.')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2s')
fig.show()

x - Valor total dos repasses por mês (tendência, velocidade e aceleração)

In [28]:
df_temp = df_e.copy()
filtro = df_temp.UF == 'AP'
df_temp = df_temp[filtro].resample(rule='M').sum(numeric_only=True)
df_temp['MEDIA MOVEL'] = df_temp['TOTAL LIQUIDO'].rolling(window=12).mean()

fig = px.line(data_frame=df_temp,  y=['TOTAL LIQUIDO', 'MEDIA MOVEL'], x=df_temp.index, title='Total de repasses efetuados aos estados por mês de 01-2007 a 09-2023')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()

x - Estacionariedade e ruido

In [12]:
df_temp = df_e.copy()
df_temp = df_temp.resample(rule='m').sum(numeric_only=True)
df_temp['DIFERENCIAÇÃO'] = df_temp['TOTAL LIQUIDO'].diff(1)

fig = px.bar(data_frame=df_temp, x=df_temp.index, y=['DIFERENCIAÇÃO'])
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()

x - Sazonalidade

In [18]:
df_temp = df_e.copy()
df_temp = df_temp.resample(rule='m').sum(numeric_only=True)
df_temp['MEDIA MOVEL'] = df_temp['TOTAL LIQUIDO'].rolling(window=7).mean()
df_temp['SAZONALIDADE'] = df_temp['TOTAL LIQUIDO'].diff(1)
df_temp = df_temp['SAZONALIDADE'].groupby(df_temp.index.month).mean()

fig = px.bar(data_frame=df_temp, x=df_temp.index, y=['SAZONALIDADE'])
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()

x - Média anual dos repasses por estados

In [16]:
df_temp = df_e.copy()
df_temp = df_temp.groupby('UF').resample(rule='Y').sum(numeric_only=True).reset_index(level=0)
df_temp = df_temp.groupby('UF').mean().sort_values(by='TOTAL LIQUIDO')

fig = px.bar(data_frame=df_temp,  x='TOTAL LIQUIDO', y=df_temp.index, title='Média do total de repasses recebidos por ano para cada estado de 2007 a 10-2023')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y unified', xaxis_tickformat=',.2f')
fig.show()

x - Média anual dos repasses por fonte

In [21]:
df_temp = df_e.copy()
df_temp = df_temp.groupby('REPASSE').resample(rule='Y').sum(numeric_only=True).reset_index(level=0)
df_temp = df_temp.groupby('REPASSE').mean().sort_values(by='TOTAL LIQUIDO')


fig = px.pie(data_frame=df_temp,  names=df_temp.index, values='TOTAL LIQUIDO', opacity=0.8, hole=0.5, title='Média do total de repasses recebidos por ano para cada fonte de 2007 a 09-2023')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y unified', xaxis_tickformat=',.2f')
fig.show()

## QUADRO DO PERÍODO ATUAL


x - Total de repasses por mês de 2023

In [75]:
df_temp = df_e.copy()
filter = (df_temp.index.year >= 2023) & (df_temp.index.month <= 10)
df_temp = df_temp[filter].resample(rule='MS').sum(numeric_only=True)
display(df_temp.head())

fig = px.line(data_frame=df_temp, x=df_temp.index, y='TOTAL LIQUIDO')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y unified', yaxis_tickformat=',.2f')
fig.show()

,TOTAL AJUSTE,TOTAL LIQUIDO
COMPETÊNCIA,,
2023-01-01,0.00,"10,429,990,041.45"
2023-02-01,0.00,"8,587,795,911.91"
2023-03-01,0.00,"7,853,008,524.39"
2023-04-01,"-10,054,260.31","7,439,835,164.80"
2023-05-01,"437,251,746.72","9,446,093,813.64"


In [37]:
df_temp = df_e.copy()
filter = (df_temp.UF == 'AP') & (df_temp.index.year >= 2023) & (df_temp.index.month <= 10)
df_temp = df_temp[filter].resample(rule='MS').sum(numeric_only=True)
df_temp['SOMA CUMULATIVA'] = df_temp['TOTAL LIQUIDO'].cumsum()
df_temp.head()

fig = px.bar(data_frame=df_temp,  y=['SOMA CUMULATIVA'], x=df_temp.index, title='Total de repasses efetuados aos estados por mês de 01-2007 a 10-2023')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()

x - Ranking dos estados com maior valor total de repasses no período

In [82]:
df_temp = df.copy().set_index(keys='COMPETÊNCIA')
df_temp = df_temp.loc[(df_temp.index.year >= 2023) & (df_temp.index.month <= 10)]
df_temp = df_temp[df_temp.ESFERA =='Estadual']
df_temp = df_temp.groupby('UF').resample(rule='Y').sum(numeric_only=True).reset_index(level=0).sort_values(by='TOTAL LIQUIDO')
display(df_temp.head())

fig = px.bar(data_frame=df_temp, y='UF', x='TOTAL LIQUIDO')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()

,UF,TOTAL AJUSTE,TOTAL LIQUIDO
COMPETÊNCIA,,,
2023-12-31,RR,0.00,"543,339,405.18"
2023-12-31,AP,0.00,"718,868,718.32"
2023-12-31,AC,0.00,"791,268,323.83"
2023-12-31,SE,-6.82,"810,731,825.81"
2023-12-31,AL,"2,044,196.24","883,863,212.29"


x - Ranking do valor total de repasses por fonte

In [36]:
df_temp = df_e.copy()
filter = (df_temp.index.year == 2023) & (df_temp.index.month <= 10)
df_temp = df_temp[filter].groupby('REPASSE').sum(numeric_only=True).loc[['COUN_VAAF', 'COUN_VAAR', 'COUN_VAAT', 'FPE', 'FPM', 'ICMS', 'IPI', 'IPVA', 'ITCMD', 'ITR'], :].sort_values(by='TOTAL LIQUIDO', ascending=False)

fig = px.bar(data_frame=df_temp, x=df_temp.index.get_level_values(0), y='TOTAL LIQUIDO')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='x unified', yaxis_tickformat=',.2f')
fig.show()